# Thiessen_Diffusion

### Goal:
Update $V_{ij}^{t}$ for all US counties.

### Preparation: 
Create Thiessen polygons for all known __675__ airports in the US (in `Voronoi.mxd`).
1. Make sure the airports layer contains IATA code and cooridates. 
* `Create Thiessen Polygons` for `US_airports_675` to create `US_airports_Thiessen` (Output Fields: ALL).
* `Dissolve` `us_states` to create `US_Boundary` as the mask.
* `Clip` `US_airports_Thiessen` with `US_Boundary` to make sure all Thiessen polygons are within the US. Output: `US_airports_Thiessen_Clip`.
* Calculate geometry (`ThiessenAreaKM2`) for each Thiessen polygon.
* `Intersect` `US_airports_Thiessen_Clip` and `us_states` to get `Thiessen_County_Intersect`.
* Calcuate geometry (`IntersectAreaKM2`) for each polygon in `Thiessen_County_Intersect`.
* Calcuate percentage of intersected polygon to the airport Thiessen polygon (`ThiessenAreaPct = [IntersectAreaKM2] * 100/ [ThiessenAreaKM2]`).
* Export `Thiessen_County_Intersect` as `Thiessen_County_Intersect_Pct.csv`

### Method:
Diffusing international incoming travel volume ( $V_{ij}^{t}$) to all neighboring counties. 

In [90]:
# environment setting
import datetime
t = datetime.datetime.now()
import pandas as pd
year = 2019
year_pop = 'pop2019' # we use 2015 data for 2015-2019
year_iata = 2017 # we use 2017 IATA data for 2018 and 2019
folder = r'C:\Users\Ensheng\Desktop\mapping\Voronoi\\'
pd.set_option("display.max_rows", 999)

#### Import original $V_{ij}^{t}$

In [91]:
# IATA data
in_table = r'C:\Users\Ensheng\Desktop\mapping\IATA\flow_XY.csv'
# Note: CSL and SBP are the same airport. CSL -> SBP (Airport count 676 -> 675)
df_iata = pd.read_csv(in_table)
df_iata = df_iata.loc[df_iata['year'] == year_iata] # slice for certain year
# note: FIPS means the state where the airport (IATA) is located. One airport (IATA) has only one associated state (FIPS).
df_iata = df_iata[['ISO', 'Code', 'FIPS', 'paxVolume']]
print(len(df_iata))
df_iata.head(5)

38109


,ISO,Code,FIPS,paxVolume
332862,MEX,MSL,1033,2
332863,CHE,MSL,1033,2
332864,DEU,DHN,1045,699
332865,CAN,DHN,1045,577
332866,KOR,DHN,1045,348


In [92]:
print("Warning: " + str(len(df_iata.loc[df_iata['Code'].isnull()])) + " airport(s) missing info.")

#### Update incoming travel volume data

In [93]:
# Thiessen data
in_table = folder + 'Thiessen_County_Intersect_Pct.csv'
df_tpct = pd.read_csv(in_table)
# note: FIPS_1 means all states within an airport Thiessen polygon. One airport (Code) has at least one associated state (FIPS_1).
# The sum of ThiessenAreaPct for the same airport should be 100%.
df_tpct = df_tpct[['Code', 'FIPS_1', 'ThiessenAreaPct']]
print(len(df_tpct))
df_tpct.sort_values(by='Code').head(15)

7137


,Code,FIPS_1,ThiessenAreaPct
6071,ABE,42095,14.276210
6070,ABE,42089,11.460134
6069,ABE,42011,14.898813
6068,ABE,42077,13.235343
6067,ABE,42017,8.398984
6066,ABE,42091,5.618685
6065,ABE,42103,0.115441
6064,ABE,34041,8.097077
6063,ABE,42107,10.020279
6061,ABE,42025,10.374824


In [94]:
# diffuse the travel volume to each county (make sure there is no null values after the left join)
df_temp = pd.merge(df_iata, df_tpct, how='left', on='Code')
df_temp.loc[df_temp['FIPS_1'].isnull()]

,ISO,Code,FIPS,paxVolume,FIPS_1,ThiessenAreaPct


In [95]:
df_temp['travelVolume'] = df_temp['paxVolume'] * df_temp['ThiessenAreaPct'] / 100
print(len(df_temp))
df_temp.sort_values(by=['ISO','Code']).head(15)

481452


,ISO,Code,FIPS,paxVolume,FIPS_1,ThiessenAreaPct,travelVolume
326529,ABW,ABE,42077,428,42025,10.374824,44.404246
326530,ABW,ABE,42077,428,34019,3.324779,14.230053
326531,ABW,ABE,42077,428,42107,10.020279,42.886796
326532,ABW,ABE,42077,428,34041,8.097077,34.655489
326533,ABW,ABE,42077,428,42103,0.115441,0.494089
326534,ABW,ABE,42077,428,42091,5.618685,24.047973
326535,ABW,ABE,42077,428,42017,8.398984,35.947651
326536,ABW,ABE,42077,428,42077,13.235343,56.647267
326537,ABW,ABE,42077,428,42011,14.898813,63.766919
326538,ABW,ABE,42077,428,42089,11.460134,49.049374


In [96]:
df = df_temp.groupby(['ISO','FIPS_1'])['travelVolume'].sum().reset_index()
# update df_iata with travel volume for more counties
df["FIPS"] = df["FIPS_1"]
df["paxVolume"] = df["travelVolume"]
df = df[["FIPS","ISO","paxVolume"]]
df.head(5)

,FIPS,ISO,paxVolume
0,1001,ABW,2.941992
1,1003,ABW,18.354891
2,1005,ABW,2.273760
3,1007,ABW,88.149201
4,1009,ABW,83.840063


In [97]:
# environment setting
import datetime
t = datetime.datetime.now()
import pandas as pd
year = 2019
year_pop = 'pop2019' # we use 2015 data for 2015-2019
year_iata = 2017 # we use 2017 IATA data for 2018 and 2019
folder = r'C:\Users\Ensheng\Desktop\mapping\diffusion_model\\'
pd.set_option("display.max_rows", 999)

#### Import world population

In [98]:
# ref: http://worldpopulationreview.com/countries/
in_table = r'C:\Users\Ensheng\Desktop\mapping\diffusion_model\world_population.csv'
df_pop = pd.read_csv(in_table)
print(len(df_pop))
df_pop.head(5)

230


,Rank,name,pop2019,pop2018,GrowthRate,area,Density
0,39,Afghanistan,37209.007,36373.176,1.022979,652230.0,57.048905
1,138,Albania,2938.428,2934.363,1.001385,28748.0,102.213302
2,34,Algeria,42679.018,42008.054,1.015972,2381741.0,17.919252
3,208,American Samoa,55.727,55.679,1.000862,199.0,280.035176
4,202,Andorra,77.072,76.953,1.001546,468.0,164.683761


In [99]:
# ref: http://worldpopulationreview.com/country-codes/
in_table = r'C:\Users\Ensheng\Desktop\mapping\diffusion_model\country_code.csv'
df_code = pd.read_csv(in_table)
print(len(df_code))
df_code.head(5)

237


,name,alpha2,alpha3,num3
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20


#### Import WHO data (exclude 2019)

In [69]:
# ref: https://www.who.int/immunization/monitoring_surveillance/burden/vpd/surveillance_type/active/measles_monthlydata/en/
in_table = r'C:\Users\Ensheng\Desktop\mapping\diffusion_model\measlescasesbycountrybymonth.xls'
df_who = pd.read_excel(in_table,sheet_name='WEB')
df_who = df_who.loc[df_who['Year'] == year]
print(len(df_who))
df_who.head(3)

194


,Region,ISO3,Country,Year,January,February,March,April,May,June,July,August,September,October,November,December
8,AFR,AGO,Angola,2019,40.0,49.0,229.0,440.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,AFR,BDI,Burundi,2019,0.0,6.0,34.0,12.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,AFR,BEN,Benin,2019,290.0,154.0,45.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
col_list= list(df_who)
col_list.remove('Year')
df_who['Total'] = df_who[col_list].sum(axis=1)
print(len(df_who))
df_outbreak_raw = df_who[['ISO3','Country','Total']]
df_outbreak_raw.head(3)

194


,ISO3,Country,Total
8,AGO,Angola,758.0
17,BDI,Burundi,52.0
26,BEN,Benin,493.0


#### WHO 2019 Susptected Data (Optional)

In [100]:
# ref: https://www.who.int/immunization/monitoring_surveillance/burden/vpd/surveillance_type/active/measles_monthlydata/en/
in_table = r'C:\Users\Ensheng\Desktop\mapping\diffusion_model\WHO_2019_Suspected.xlsx'
df_who = pd.read_excel(in_table)
print(len(df_who))
df_outbreak_raw = df_who.fillna(0)
df_outbreak_raw.head(3)

194


,Country,ISO3,Total
0,Algeria,DZA,0.0
1,Angola,AGO,854.0
2,Benin,BEN,512.0


In [112]:
df_pop3 = pd.merge(df_pop, df_code, how='left', left_on='name',right_on='name')
df_outbreak = pd.merge(df_outbreak_raw, df_pop3, how='left', left_on='ISO3',right_on='alpha3')
print(len(df_outbreak))
df_outbreak = df_outbreak[['alpha3', 'ISO3','Country', 'Total', year_pop]]
print(str(len(df_outbreak_raw) - df_outbreak.alpha3.notnull().sum()) + " row(s) have NaN as ISO 3 (alpha3).")
df_outbreak.sort_values(by='alpha3').head(5)

194
0 row(s) have NaN as ISO 3 (alpha3).


,alpha3,ISO3,Country,Total,pop2019
82,AFG,AFG,Afghanistan,169.0,37209.007
1,AGO,AGO,Angola,854.0,31787.566
103,ALB,ALB,Albania,508.0,2938.428
104,AND,AND,Andorra,0.0,77.072
101,ARE,ARE,United Arab Emirates,66.0,9682.088


#### Import $V_{ij}^{t}$

In [73]:
# IATA data
in_table = r'C:\Users\Ensheng\Desktop\mapping\IATA\flow_XY.csv'
df_iata = pd.read_csv(in_table)
df_iata = df_iata.loc[df_iata['year'] == year_iata] # slice for certain year
df_iata = df_iata[['FIPS', 'ISO', 'paxVolume']]
print(len(df_iata))
df_iata.head(5)

38109


,FIPS,ISO,paxVolume
332862,1033,MEX,2
332863,1033,CHE,2
332864,1045,DEU,699
332865,1045,CAN,577
332866,1045,KOR,348


In [102]:
df_iata = df
print(len(df_iata))
df_iata.head(5)

359071


,FIPS,ISO,paxVolume
0,1001,ABW,2.941992
1,1003,ABW,18.354891
2,1005,ABW,2.273760
3,1007,ABW,88.149201
4,1009,ABW,83.840063


#### Import $NME_{j}^{t}$ and $P_{j}^{t}$

In [103]:
in_table = r'C:\Users\Ensheng\Desktop\mapping\diffusion_model\ModelInputOutputAll 4_23.csv'
df_nme = pd.read_csv(in_table)
print(len(df_nme))
df_nme.head(5)

3142


,County Name,State,FIPS,2015_NME,2016_NME,State_Avg_NME,Population,Static,Year2011,Year2012,Year2013,Year2014,Year2015,Year2016,Year2017,Year2018,Year2019
0,Autauga,Alabama,1001,NaN,NaN,0.006,55504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baldwin,Alabama,1003,NaN,NaN,0.006,212628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Barbour,Alabama,1005,NaN,NaN,0.006,25270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bibb,Alabama,1007,NaN,NaN,0.006,22668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Blount,Alabama,1009,NaN,NaN,0.006,58013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
df_nme['County'] = df_nme['County Name'] + ', ' + df_nme['State']

In [105]:
df_nme.loc[df_nme["2016_NME"].notnull(), 'FIPS_NME'] = df_nme['2016_NME']
df_nme.loc[(df_nme["FIPS_NME"].isnull()) & (df_nme["2015_NME"].notnull()), 'FIPS_NME'] = df_nme['2015_NME']
df_nme.loc[(df_nme["FIPS_NME"].isnull()) & (df_nme["State_Avg_NME"].notnull()), 'FIPS_NME'] = df_nme['State_Avg_NME']

In [106]:
df_nme = df_nme[['FIPS','County','2016_NME','2015_NME','State_Avg_NME','FIPS_NME','Population']]
print("No NME for the following counties:")
df_nme.loc[df_nme['FIPS_NME'].isnull()]

No NME for the following counties:


,FIPS,County,2016_NME,2015_NME,State_Avg_NME,FIPS_NME,Population
3119,56001,"Albany, Wyoming",NaN,NaN,NaN,NaN,38332
3120,56003,"Big Horn, Wyoming",NaN,NaN,NaN,NaN,11906
3121,56005,"Campbell, Wyoming",NaN,NaN,NaN,NaN,46242
3122,56007,"Carbon, Wyoming",NaN,NaN,NaN,NaN,15303
3123,56009,"Converse, Wyoming",NaN,NaN,NaN,NaN,13809
3124,56011,"Crook, Wyoming",NaN,NaN,NaN,NaN,7410
3125,56013,"Fremont, Wyoming",NaN,NaN,NaN,NaN,39803
3126,56015,"Goshen, Wyoming",NaN,NaN,NaN,NaN,13378
3127,56017,"Hot Springs, Wyoming",NaN,NaN,NaN,NaN,4696
3128,56019,"Johnson, Wyoming",NaN,NaN,NaN,NaN,8476


#### Calculate $r_{ij}^{t}$

In [113]:
df_temp = pd.merge(df_iata, df_outbreak, how='left', left_on='ISO',right_on='alpha3')
df_factors = pd.merge(df_temp, df_nme, how='left', left_on='FIPS',right_on='FIPS')
df_factors.tail(5)

,FIPS,ISO,paxVolume,alpha3,ISO3,Country,Total,pop2019,County,2016_NME,2015_NME,State_Avg_NME,FIPS_NME,Population
359066,56025,ZWE,0.419465,ZWE,ZWE,Zimbabwe,57.0,17297.495,"Natrona, Wyoming",NaN,NaN,NaN,NaN,79547
359067,56031,ZWE,0.000678,ZWE,ZWE,Zimbabwe,57.0,17297.495,"Platte, Wyoming",NaN,NaN,NaN,NaN,8562
359068,56037,ZWE,0.003640,ZWE,ZWE,Zimbabwe,57.0,17297.495,"Sweetwater, Wyoming",NaN,NaN,NaN,NaN,43534
359069,56041,ZWE,1.656888,ZWE,ZWE,Zimbabwe,57.0,17297.495,"Uinta, Wyoming",NaN,NaN,NaN,NaN,20495
359070,56043,ZWE,0.000027,ZWE,ZWE,Zimbabwe,57.0,17297.495,"Washakie, Wyoming",NaN,NaN,NaN,NaN,8064


In [114]:
# rename and reorder col.
df_factors.loc[:,('FIPS_Pop')] = df_factors['Population']
df_factors.loc[:,('ISO_Case')] = df_factors['Total']
df_factors.loc[:,('ISO_Pop')] = df_factors[year_pop]
df_factors = df_factors[['FIPS','County','FIPS_NME','FIPS_Pop','ISO','Country','ISO_Case','ISO_Pop','paxVolume']]
print(len(df_factors))
df_factors.head(5)

359071


,FIPS,County,FIPS_NME,FIPS_Pop,ISO,Country,ISO_Case,ISO_Pop,paxVolume
0,1001,"Autauga, Alabama",0.006,55504,ABW,NaN,NaN,NaN,2.941992
1,1003,"Baldwin, Alabama",0.006,212628,ABW,NaN,NaN,NaN,18.354891
2,1005,"Barbour, Alabama",0.006,25270,ABW,NaN,NaN,NaN,2.273760
3,1007,"Bibb, Alabama",0.006,22668,ABW,NaN,NaN,NaN,88.149201
4,1009,"Blount, Alabama",0.006,58013,ABW,NaN,NaN,NaN,83.840063


In [115]:
# slice
df_factors = df_factors.loc[df_factors['ISO_Case'].notnull()]
print(len(df_factors))
df_factors = df_factors.loc[df_factors['paxVolume'].notnull()]
print(len(df_factors))

315695
315695


#### Calculate $r_{j}^{t}$

In [116]:
df_factors['Route_Risk'] = (df_factors['ISO_Case'] / df_factors['ISO_Pop']) * df_factors['paxVolume'] * df_factors['FIPS_NME'] * df_factors['FIPS_Pop']

In [117]:
df_risk = df_factors.groupby(['FIPS','County'])['Route_Risk'].sum().reset_index()
df_risk.loc[:,('FIPS_RawRisk')] = df_risk['Route_Risk']
df_risk.head(5)

,FIPS,County,Route_Risk,FIPS_RawRisk
0,1001,"Autauga, Alabama",4013.960943,4013.960943
1,1003,"Baldwin, Alabama",113459.294852,113459.294852
2,1005,"Barbour, Alabama",2033.738643,2033.738643
3,1007,"Bibb, Alabama",14822.806691,14822.806691
4,1009,"Blount, Alabama",40510.195240,40510.195240


#### Normalize and list the Top 25

In [118]:
# import county seats
# ref: https://en.wikipedia.org/wiki/List_of_the_most_populous_counties_in_the_United_States
in_table = r'C:\Users\Ensheng\Desktop\mapping\diffusion_model\County_Seat.xlsx'
df_seat = pd.read_excel(in_table)
print(len(df_seat))
df_seat.head(3)

100


,County,City
0,"Los Angeles, California",Los Angeles
1,"Cook, Illinois",Chicago
2,"Harris, Texas",Houston


In [119]:
highest_risk = df_risk['FIPS_RawRisk'].max()
df_risk['Risk'] = df_risk['FIPS_RawRisk'] / highest_risk
df_risk['FIPS_Rank'] = df_risk['Risk'].rank(ascending=False)
df_risk = pd.merge(df_risk, df_seat, how='left', left_on='County',right_on='County')
df_risk = df_risk[['FIPS','County','City','FIPS_RawRisk','Risk','FIPS_Rank']]
df_risk = df_risk.sort_values('Risk',ascending = False).reset_index()
df_risk.head(50)

,index,FIPS,County,City,FIPS_RawRisk,Risk,FIPS_Rank
0,169,6037,"Los Angeles, California",Los Angeles,8.137663e+09,1.000000,1.0
1,320,12086,"Miami-Dade, Florida",Miami,4.035967e+09,0.495961,2.0
2,519,17031,"Cook, Illinois",Chicago,3.593620e+09,0.441603,3.0
3,92,4013,"Maricopa, Arizona",Phoenix,2.120860e+09,0.260623,4.0
4,284,12011,"Broward, Florida",Fort Lauderdale,1.599025e+09,0.196497,5.0
5,1648,36081,"Queens, New York","Queens, NYC",1.455652e+09,0.178878,6.0
6,461,15003,"Honolulu, Hawaii",Honolulu,1.312220e+09,0.161253,7.0
7,1573,34025,"Monmouth, New Jersey",NaN,1.244377e+09,0.152916,8.0
8,1535,32003,"Clark, Nevada",Las Vegas,1.128968e+09,0.138734,9.0
9,1631,36047,"Kings, New York","Brooklyn, NYC",9.540837e+08,0.117243,10.0


In [121]:
result = df_risk
output_csv = folder + 'MeaslesRisk_US_' +  str(year) + '_raw_' + t.strftime('%m%d%y%H%M') + '.csv'
result.to_csv(output_csv, index=False, encoding='utf-8')

In [120]:
df_complete = pd.merge(df_factors, df_risk , how='left', left_on='FIPS',right_on='FIPS')
df_complete = df_complete.sort_values(by=['Risk','Route_Risk'], ascending=False)
df_complete['Route_Rank'] = df_complete.groupby('FIPS_Rank')['Route_Risk'].rank(ascending=False,method='dense')
df_complete = df_complete.rename(index=str, columns={"County_x": "County"})
df_complete = df_complete.drop(columns=['County_y'])
print(len(df_complete))
df_complete.head(10)

315695


,FIPS,County,FIPS_NME,FIPS_Pop,ISO,Country,ISO_Case,ISO_Pop,paxVolume,Route_Risk,index,City,FIPS_RawRisk,Risk,FIPS_Rank,Route_Rank
196053,6037,"Los Angeles, California",0.006,10163507,MEX,Mexico,1122.0,132328.035,1.894564e+06,9.795918e+08,169,Los Angeles,8.137663e+09,1.0,1.0,1.0
298756,6037,"Los Angeles, California",0.006,10163507,UKR,Ukraine,34251.0,43795.220,1.695928e+04,8.088141e+08,169,Los Angeles,8.137663e+09,1.0,1.0,2.0
157672,6037,"Los Angeles, California",0.006,10163507,JPN,Japan,2412.0,126854.745,5.352442e+05,6.206080e+08,169,Los Angeles,8.137663e+09,1.0,1.0,3.0
148166,6037,"Los Angeles, California",0.006,10163507,ISR,Israel,614.0,8583.916,1.318455e+05,5.750998e+08,169,Los Angeles,8.137663e+09,1.0,1.0,4.0
169755,6037,"Los Angeles, California",0.006,10163507,KOR,Republic of Korea,1139.0,51339.238,3.385227e+05,4.579910e+08,169,Los Angeles,8.137663e+09,1.0,1.0,5.0
283844,6037,"Los Angeles, California",0.006,10163507,THA,Thailand,3428.0,69306.160,1.363737e+05,4.113339e+08,169,Los Angeles,8.137663e+09,1.0,1.0,6.0
242227,6037,"Los Angeles, California",0.006,10163507,PHL,Philippines,1812.0,108106.310,3.716872e+05,3.799090e+08,169,Los Angeles,8.137663e+09,1.0,1.0,7.0
55016,6037,"Los Angeles, California",0.006,10163507,CHN,China,10789.0,1420062.022,7.697130e+05,3.566133e+08,169,Los Angeles,8.137663e+09,1.0,1.0,8.0
103299,6037,"Los Angeles, California",0.006,10163507,FRA,France,628.0,65480.710,3.444647e+05,2.014586e+08,169,Los Angeles,8.137663e+09,1.0,1.0,9.0
176038,6037,"Los Angeles, California",0.006,10163507,LBN,Lebanon,598.0,6065.922,2.889320e+04,1.736980e+08,169,Los Angeles,8.137663e+09,1.0,1.0,10.0


In [122]:
result = df_complete
output_csv = folder + 'MeaslesRisk_US_' +  str(year) + '_raw_route_' + t.strftime('%m%d%y%H%M') + '.csv'
result.to_csv(output_csv, index=False, encoding='utf-8')